In [2]:
import json
import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime

In [ ]:
# ETAPE 1 connexion à l'API + récupération offres
"""
comme je n'arrivais pas à bien séparer les offres les unes des autres, j'ai choisi 1 résultat par page de manière temporaire 
à  voir ensuite comment améliorer performance
"""

def create_url(start_url="http://api.adzuna.com/v1/api/jobs/"):
    """
    concactène l'adresse url à partir des paramètres choisis
    comment améliorer pour que les paramètres soient correctement passés ?
    """
    url_list = []
    for page_number in range(1, 3):
        #start_url = "http://api.adzuna.com/v1/api/jobs/"
        country = "fr"
        part0_url = "/search/"
        number_page = str(page_number) 
        part1_url = "?app_id="
        api_id = "37fe08af"
        part2_url = "&app_key="
        api_key = "17c1369ea8fb68bc48f834aebc0bec53"
        part3_url = "&results_per_page="
        results_per_page = str(1)
        part4_url = "&what_and="
        what_and = "data%20engineer"
        part5_url = "&max_days_old="
        max_days_old = str(30)
        end_url = "&content-type=application/json"
        full_url = f"{start_url}{country}{part0_url}{number_page}{part1_url}{api_id}{part2_url}{api_key}{part3_url}{results_per_page}{part4_url}{what_and}{part5_url}{max_days_old}{end_url}"
        url_list.append(full_url)
    return url_list


def scrape_urls(url):
    """
    lance la requête sur l'api Adzuna pour chacunes des url crééés
    si le status_code est incorrect, il s'affiche et l'url liée s'affiche
    """
    try:
        response = requests.get(url)
    except Exception as response.status_code NOT 200:
        print("The request for {} was not successful : status {}".format(url, response.status_code))
        return None
    return response
